# Text to Multiclass Explanation: Language Modelling Example

This notebook demostrates how to get explanations for the top-k next words generated by a language model. In this demo, we use the pretrained gpt2 model provided by hugging face (https://huggingface.co/gpt2) to predict the top-k next words. By looking at the top-k next words, we treat them as k separate classes and then learn the explanations for each of this k words. We thereby are able to explain the contribution of words in the input that are responsible for the liklihood of the top-k next words to be predicted. 

In [1]:
import numpy as np
from transformers import AutoTokenizer, AutoModelForCausalLM
import shap
import torch

### Load model and tokenizer

In [2]:
tokenizer = AutoTokenizer.from_pretrained('gpt2', use_fast=True)
model = AutoModelForCausalLM.from_pretrained('gpt2').cuda()

We next wrap the model with the TopKLM model which extracts the log odds of the top-k next words and also create a Text masker by initializing it with the mask_token = "..." and set collapse_mask_token = True, which is used for infilling text during perturbation of the inputs.

In [3]:
wrapped_model = shap.models.TopKLM(model, tokenizer, k=100)
masker = shap.maskers.Text(tokenizer, mask_token = "...", collapse_mask_token=True)

Using pad_token, but it is not set yet.


### Define data

Here we set the initial text for which we want the gpt2 model to predict the next word

In [4]:
s = ["In a shocking finding, scientists discovered a herd of unicorns living in a"]

### Create explainer object

In [5]:
explainer = shap.Explainer(wrapped_model,masker)

explainers.Partition is still in an alpha state, so use with caution...


### Compute SHAP values

In [6]:
shap_values = explainer(s)

### Visualize the SHAP values across the input sentence for the top-k next words

We can now see the top-k next words predicted by gpt2 under "Output Text" in the viz plot below and hover over each of the token to understand which words in the input sentence are driving the generation of the particular output word to be predicted

In [7]:
shap.plots.text(shap_values)

,In a,shocking,finding,",",scientists discovered,a,herd,of unicorns living,in,a
cave,0.263,-0.035,-0.086,-0.628,0.538,0.227,0.022,1.809,2.772,4.778
forest,0.407,-0.262,-0.234,-0.124,-0.13,0.363,0.456,1.309,2.311,4.953
small,0.143,-0.004,0.134,-0.387,-0.45,0.301,0.126,0.216,0.927,4.738
desert,0.002,-0.053,0.121,-0.449,0.429,0.373,0.957,1.048,2.298,3.785
tiny,0.088,0.16,0.178,-0.338,0.302,0.134,-0.015,0.784,0.558,4.962
"""",-0.432,0.148,0.126,0.286,0.421,-0.071,-0.26,0.309,0.19,0.111
remote,0.163,-0.222,-0.158,-0.079,0.597,0.564,0.137,0.305,1.968,4.032
zoo,0.565,0.47,0.369,-0.311,0.245,0.317,0.612,1.552,1.037,3.856
tree,0.203,-0.423,-0.372,-0.238,0.2,0.224,-0.065,2.012,1.078,3.81
field,0.641,-0.5,-0.405,-0.066,-0.355,0.455,0.372,-0.373,1.954,3.841
